In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported

from trl import SFTTrainer
from transformers import TrainingArguments
from tuning.data.train_dataset import get_train_dataset
from tuning.training.config_training import ModelLoadConfig, LoraConfig, SFTRunConfig, PTRunConfig, DPOTrainingConfig, TrainingArgumentsConfig, PassAtKConfig, sft_batch_size, effective_batch_size
from tuning.training.perplexity_callback import PerplexityStoppingCallback
from tuning.training.passk_callback import PassAtKStoppingCallback
from tuning.utils.utils import apply_chat_template, chat_template_func
import json
import sys
from datasets import load_from_disk
from typing import List, Optional, Union
from pathlib import Path
from tuning.config import DATASETS_DIR, HF_MODEL_MAP
import os
from tuning.training.config_training import DatasetConfig, SFTRunConfig
from tuning.config import MODELS_DIR
from tuning.training.sft_training import train_model_sft
from tuning.training.dpo_training import train_model_dpo
import subprocess
import importlib

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 01-26 02:33:02 [__init__.py:244] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This m

/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning


In [ ]:
import importlib
import tuning.training.passk_callback
importlib.reload(tuning.training.passk_callback)
from tuning.training.passk_callback import PassAtKStoppingCallback
from tuning.training.dpo_training import train_model_dpo
importlib.reload(tuning.training.dpo_training)
from tuning.training.dpo_training import train_model_dpo

In [2]:
MODEL = "llama3-8B"
total_train_size = 8192  # 29980
perplexity_thresholds = [7.0,6.0, 5.75, 5.5, 5.25, 5.0, 4.75, 4.5, 4.25,4.0, 3.9, 3.8, 3.7, 3.6,3.55,3.5,3.45,3.4,3.35,3.3, 3.25, 3.2, 3.15, 3.1]
perplexity_thresholds = [7.0, 6.0]

In [5]:
dataset_config = DatasetConfig(
    dataset = "tuluif",
    dataset_type = "sft",
    train_size = total_train_size, # 29980
)

run_config = SFTRunConfig(
    dataset_config = dataset_config,
    model_name_hf = HF_MODEL_MAP[MODEL],  # Use HuggingFace model name, not local path
    model_name = MODEL,  # Base model name for output directory construction
    do_training=True,
    do_inference=False,
    do_evaluation=False,
)
passk_config = PassAtKConfig( # this is just to dynamically view the pass@1 of ifeval
    target_pass_at_k=[0.1, 0.2, 0.3,0.4,0.5,0.6],
    k_values=[1],
    n_samples=1,
    num_prompts=100,
    temperature=0.7,
    strict=True,
    enabled=True,
)

lora_config = LoraConfig()
model_load_config = ModelLoadConfig()
model_load_config.max_seq_length = 4096
training_args = TrainingArgumentsConfig()

In [7]:
model, tokenizer, trainer, callbacks = train_model_sft(
    run_config = run_config,
    lora_config = lora_config,
    model_load_config = model_load_config,
    training_args = training_args,
    passk_config = passk_config
)

Getting train dataset for run config: llama3-8B_sft-tuluif-8192
Checking for dataset at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/sft-tuluif-8192
Dataset already exists at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/sft-tuluif-8192
Sampled dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 8192
    })
    test: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 200
    })
})
Example training row: {'id': 'personas_IF_9u3mcrurksv7hypq3xlppyba', 'prompt': 'Compile a detailed summary of the most recent case discussed on the podcast. Include the title of the episode wrapped in double angular brackets, i.e. <<title>>, and quote at least one statement made by the host during the discussion.', 'messages': [{'content': 'You are a helpful assistant who is an expert at responding to prompts by carefully followin

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints', 'text'],
        num_rows: 8192
    })
    test: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints', 'text'],
        num_rows: 200
    })
})
{'id': 'personas_IF_9u3mcrurksv7hypq3xlppyba', 'prompt': 'Compile a detailed summary of the most recent case discussed on the podcast. Include the title of the episode wrapped in double angular brackets, i.e. <<title>>, and quote at least one statement made by the host during the discussion.', 'messages': [{'content': 'You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions', 'role': 'system'}, {'content': 'Compile a detailed summary of the most recent case discussed on the podcast. Include the title of the episode wrapped in double angular brackets, i.e. <<title>>, and quote at least one statement made by the host during the discussion.', 'role': 'user'}, {'content': "I'm

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,192 | Num Epochs = 2 | Total steps = 1,024
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


[PassAtKCallback] on_train_begin: model_name=llama3-8B


...

Successfully copied all 4 files from cache to `/tmp/tmppk1lqiu6`


Unsloth: Merging weights into 16bit: 100%|███████████████████████████████████| 4/4 [00:50<00:00, 12.73s/it]


Unsloth: Merge process complete. Saved to `/tmp/tmppk1lqiu6`
[PassAtKCallback] Moving training model to CPU...
[PassAtKCallback] Loading model with vLLM from /tmp/tmppk1lqiu6...
LLM UTILISATION IS 0.8
INFO 01-26 03:08:28 [config.py:841] This model supports multiple tasks: {'reward', 'classify', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 01-26 03:08:28 [config.py:1472] Using max model len 131072
INFO 01-26 03:08:28 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-26 03:08:47 [__init__.py:244] Automatically detected platform cuda.
INFO 01-26 03:08:52 [core.py:526] Waiting for init message from front-end.
INFO 01-26 03:08:52 [core.py:69] Initializing a V1 LLM engine (v0.9.2) with config: model='/tmp/tmppk1lqiu6', speculative_config=None, tokenizer='/tmp/tmppk1lqiu6', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=13107

Process EngineCore_0:
Traceback (most recent call last):
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/python/3.11.5/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/python/3.11.5/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/vllm/v1/engine/core.py", line 590, in run_engine_core
    raise e
  File "/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/vllm/v1/engine/core.py", line 577, in run_engine_core
    engine_core = EngineCoreProc(*args, **kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/vllm/v1/engine/core.py", line 404, in __init__
    super().__init__(vllm_config, ex

RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}

In [ ]:
ppl_callback = callbacks[-1]
metadata_file = ppl_callback.metadata_path
checkpoints = []
with open(metadata_file, "r") as f:
    for line in f:
        checkpoints.append(json.loads(line))
print(checkpoints)


[{'threshold_type': 'perplexity', 'threshold_value': 6.0, 'global_step': 40, 'checkpoint_path': '/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-8B_ppl-6.00_sft-640', 'data_points_seen': 640}]


In [ ]:
import torch
del model, tokenizer, trainer
torch.cuda.empty_cache()

In [ ]:
for checkpoint in checkpoints:
    model_name = Path(checkpoint["checkpoint_path"]).name
    data = total_train_size - checkpoint["data_points_seen"] 
    model_load_config = ModelLoadConfig()
    training_args = DPOTrainingConfig()
    training_args.eval_steps = 5
    dataset_config = DatasetConfig(
        dataset = "tuluif",
        dataset_type = "pt",
        train_size = data,
    )
    sft_run_config = SFTRunConfig(
        dataset_config = DatasetConfig(
            dataset = "tuluif",
            dataset_type = "sft",
            train_size = checkpoint["data_points_seen"],
            dynamic_path = model_name
        ),
        model_name = MODEL,
        model_name_hf = HF_MODEL_MAP[MODEL], 
        task_name = "ifeval"
    )
    run_config = PTRunConfig(
        dataset_config = dataset_config,
        # model_name_hf = HF_MODEL_MAP[MODEL],  
        model_name = MODEL,  
        sft_run_config = sft_run_config,
        task_name = "ifeval",
        pft_method = "dpo",
        do_training = True
    )
    passk_config = PassAtKConfig( # this is just to dynamically view the pass@1 of ifeval
        target_pass_at_k=[1.2],
        k_values=[1],
        n_samples=1,
        num_prompts=50,
        temperature=0.7,
        strict=True,
        enabled=True,
    )
    train_model_dpo(
        run_config = run_config,
        lora_config = lora_config,
        model_load_config = model_load_config,
        training_args = training_args,
        passk_config = passk_config,
        perplexity_thresholds= [0.1] # dummy value to periodically check perplexities too
    )





Per device train batch size: 1
Getting train dataset for run config: llama3-8B_llama3-8B_ppl-6.00_sft-640_pt-tuluif-7552
Checking for dataset at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/pt-tuluif-7552
Dataset already exists at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/pt-tuluif-7552
Sampled dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model', 'system_message'],
        num_rows: 7552
    })
    test: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model', 'system_message'],
        num_rows: 200
    })
})
Example training row: {'id': 'personas_IF_dqxglsux2n8jeu59qktlnesh', 'prompt': 'Name two famous equestrian events that are part of the international jumping circuit, and format your answer by choosing one from these options: lowercase, UPPERCASE, Title 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
for checkpoint in checkpoints:
    model_name = Path(checkpoint["checkpoint_path"]).name
    data = total_train_size - checkpoint["data_points_seen"] 
    subprocess.run(["bash", "tuning/slurm/run_dpo.sh", "llama3-8B", "tuluif", str(data), "ifeval", "dpo", model_name, str(checkpoint["data_points_seen"]) ])


Running: python tuning/run_dpo.py --model 'llama3-8B' --dataset 'tuluif' --train_size '7552' --task 'ifeval' --pft_method 'dpo' --do_training


/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/project/6105902/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This m

/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 01-25 20:38:02 [__init__.py:244] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
{'model': 'llama3-8B', 'dataset': 'tuluif', 'train_size': 7552, 'task': 'ifeval', 'pft_method': 'dpo', 'dynamic_path': 'llama3-8B_ppl-6.00_sft-640', 'sft_train_size': 640, 'do_training': True, 'do_inference': False, 'do_evaluation': False}
pt-tuluif-7552
llama3-8B_llama3-8B_ppl-6.00_sft-640_pt-tuluif-7552


wandb: Currently logged in as: shougan (shougan-university-of-waterloo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /project/6105902/shougan/balance-budget/wandb/run-20260125_203807-9vfd7vvg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run llama3-8B_llama3-8B_ppl-6.00_sft-640_pt-tuluif-7552
wandb: ⭐️ View project at https://wandb.ai/shougan-university-of-waterloo/tuning
wandb: 🚀 View run at https://wandb.ai/shougan-university-of-waterloo/tuning/runs/9vfd7vvg


Per device train batch size: 1
Getting train dataset for run config: llama3-8B_llama3-8B_ppl-6.00_sft-640_pt-tuluif-7552
Checking for dataset at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/pt-tuluif-7552
Dataset already exists at /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/data/datasets/pt-tuluif-7552
Sampled dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model', 'system_message'],
        num_rows: 7552
    })
    test: Dataset({
        features: ['id', 'prompt', 'constraints', 'chosen', 'rejected', 'chonsen_model', 'rejected_model', 'system_message'],
        num_rows: 200
    })
})
Example training row: {'id': 'personas_IF_dqxglsux2n8jeu59qktlnesh', 'prompt': 'Name two famous equestrian events that are part of the international jumping circuit, and format your answer by choosing one from these options: lowercase, UPPERCASE, Title 

...

{'loss': 0.6913, 'grad_norm': 4.572940349578857, 'learning_rate': 1.736842105263158e-06, 'rewards/chosen': -0.0017172335647046566, 'rewards/rejected': -0.0059153544716537, 'rewards/accuracies': 0.4375, 'rewards/margins': 0.004198120906949043, 'logps/chosen': -309.0335388183594, 'logps/rejected': -294.1842346191406, 'logits/chosen': -1.461841106414795, 'logits/rejected': -1.497746229171753, 'epoch': 0.07}


  4%|█▍                                     | 35/944 [02:29<1:01:54,  4.09s/it]

{'loss': 0.6971, 'grad_norm': 5.176706790924072, 'learning_rate': 1.7894736842105265e-06, 'rewards/chosen': 0.003620815696194768, 'rewards/rejected': 0.010051597841084003, 'rewards/accuracies': 0.4375, 'rewards/margins': -0.006430780049413443, 'logps/chosen': -290.6890869140625, 'logps/rejected': -293.8875427246094, 'logits/chosen': -1.4535040855407715, 'logits/rejected': -1.4691929817199707, 'epoch': 0.07}


  4%|█▍                                     | 36/944 [02:33<1:02:04,  4.10s/it]

{'loss': 0.697, 'grad_norm': 4.758002758026123, 'learning_rate': 1.8421052631578948e-06, 'rewards/chosen': -0.022864796221256256, 'rewards/rejected': -0.017035674303770065, 'rewards/accuracies': 0.5625, 'rewards/margins': -0.005829118192195892, 'logps/chosen': -398.71502685546875, 'logps/rejected': -402.7589111328125, 'logits/chosen': -1.4137160778045654, 'logits/rejected': -1.4474347829818726, 'epoch': 0.08}
